# ロバスト最適化の基本

参考：

* [Robust Optimization](https://www2.isye.gatech.edu/~nemirovs/FullBookDec11.pdf) 
* [Robust optimization - A comprehensive survey](https://www.honda-ri.de/pubs/pdf/1808.pdf) 


何かのシステムを最適化することを考えましょう．
例えば工場の製造プロセスのコスト最小化や，運送業者の最適な運搬経路の計画などが相当します．
ここでのエンジニアの任務は目的関数$f(x)$を最小化するような最適化変数$x$の設計です．

このプロセスは数学的には次のような最適化問題によって表現されます：

$$
\begin{array}{ll}
\text { optimize: } & f(\mathbf{x}) \\
\text { subject to: } & g_i(\mathbf{x}) \leqslant 0, \quad i=1, \ldots, I \\
& h_j(\mathbf{x})=0, \quad j=1, \ldots, J
\end{array}
$$

この問題を最適化すること自体，しばしばNP困難だったりと難しいことがあります．
しかし，そもそもこの最適化ができたとして，それは嬉しいでしょうか？
いくつか考えるべきことがあります：

1. この関数$f$や$g$は実世界の問題やモデルを表現しますが，完全に正しい$f$や$g$をを求めることに意味はあるでしょうか？たいてい実世界は何か不確実な要素を含みます．そのため，$f, g$に精度を求めすぎることに意味はなさそうです．
2. もし完全な最適解$x^*$が求まったとして，それは実際にデプロイできるでしょうか？例えば浮動小数点の精度などの影響で，完全な$x^*$はデプロイできそうにありません．
3. 上の最適化問題は「定常」な問題を考えていますが，実世界の問題はたいてい変化します（温度や部品の摩耗など）．

上の問題の最適解は，大概の場合でモデルや解のデプロイ時に生じる小さな変化に敏感です．
そこで，ロバスト最適化では，何らかの不確実性にさらされたとしてもパフォーマンスがあまり変化しないような解を求めることを目標にします．

ここで重要になるのは，「どのようにロバスト性をモデル化するか？」と「どうやって関連する最適化問題を解くか？」の２つです．
この２つを紹介する前に，日本の田口玄一博士が考えたタグチメソッドを紹介します．一般に，品質を保証する製品の製造は次の流れで実現されるべきです：
1. システム設計：製品に関する特性を調査し，モデル化するフェーズ
2. 変数設計：製品の質が一定以上に達するように，特定の変数を最適化するフェーズ
3. Tolerance設計：ステップ２で設計された変数を調整するフェーズ

タグチメソッドは自動車のバックミラーの最適なロバスト設計などで使われてます：[Robust optimization of an automobile rearview mirror for vibration reduction](https://link.springer.com/article/10.1007/s001580100107)


## ロバスト線型計画問題

通常の線形計画問題は次の最適化問題をときます：

$$
\min_x \{c^T x + d: Ax \leq b\}
$$

ここで，$x, c \in \mathbb{R}^n$，$d\in \mathbb{R}$，$A\in \mathbb{R}^{m\times n}$，そして$b\in \mathbb{R}^m$とします．
（通常の線形計画問題では定数ベクトル$d$は無視しても問題ないですが，ロバスト最適化を考えるときはこれも考慮すると嬉しいことがあります．また，線形な等式は不等式に変形できることにも注意しましょう（[Convex Optimization](https://web.stanford.edu/~boyd/cvxbook/bv_cvxbook.pdf)のp.132）．）

簡略化のため，使用するデータ$(c, d, A, b)$はまとめて$(m+1)\times (n+1)$の行列$D$としておきましょう．

$$
D=\left[\begin{array}{c|c}
c^T & d \\
\hline A & b
\end{array}\right]
$$
とします．

まず，このデータが揺れてしまう状況を定義します．

---

**不確実線形計画問題**

次のような線形計画問題の集合を，不確実線形計画問題（$\text{LO}_\mathcal{U}$）と呼びます：

$$
\left\{\min _x\left\{c^T x+d: A x \leq b\right\}\right\}_{(c, d, A, b) \in \mathcal{U}}
$$

ここで，$\mathcal{U} \subset \mathbb{R}^{(m+1) \times(n+1)}$を不確実集合と呼びます．
この不確実集合は次の形式でかけるとします：

$$
\mathcal{U}=\{\left[\begin{array}{c|c}
c^T & d \\
\hline A & b
\end{array}\right]=\underbrace{\left[\begin{array}{c|c}
c_0^T & d_0 \\
\hline A_0 & b_0
\end{array}\right]}_{\text {nominal data $D_0$}}+\sum_{\ell=1}^L \zeta_{\ell} \underbrace{\left[\begin{array}{c|c}
c_{\ell}^T & d_{\ell} \\
\hline A_{\ell} & b_{\ell}
\end{array}\right]}_{\text {basic shifts $D_\ell$ }}: \zeta \in \mathcal{Z} \subset \mathbb{R}^L\}
$$

つまり，何らかの基本的な摂動の集合$D_1, D_2, \dots, D_\ell$があり，それがノミナルデータ$D_0$に加わるとします．

---

この不確実線形計画問題$\text{LO}_\mathcal{U}$はまだ線形計画問題の集合に過ぎません．
この集合に対して，ロバスト最適化問題を定義していきます（なぜそんなことを考えるのか？は教科書p.9参照）．そのためにいくつか定義を導入します．

---

**ロバスト実行可能解**

$$
A x \leq b \quad \forall(c, d, A, b) \in \mathcal{U}
$$

を満たす解$x$をロバスト実行可能解と呼びます．

---

---

**ロバスト実現値**

不確実集合の中で最悪ケースの実現値をロバスト実現値と呼ぶ．

$$
\widehat{c}(x)=\sup _{(c, d, A, b) \in \mathcal{U}}\left[c^T x+d\right]
$$

---

---

**Robust Counterpart**

次の問題を，$\text{LO}_\mathcal{U}$のRobust Counterpartと呼ぶ．

$$
\min _x\left\{\widehat{c}(x)=\sup _{(c, d, A, b) \in \mathcal{U}}\left[c^T x+d\right]: A x \leq b \forall(c, d, A, b) \in \mathcal{U}\right\}
$$

つまりこれは，ロバスト実行可能解の中から，最もロバスト実現値が小さくなる解を探しています．

**等価な形式**

このRobust Counterpartは次の問題と等価であることに注意しましょう．

$$
\left.\min _{x, t}\left\{t: \begin{array}{rl}
c^T x-t & \leq-d \\
A x & \leq b
\end{array}\right\} \forall(c, d, A, b) \in \mathcal{U}\right\}
$$

このようにスラック変数$t$を導入すると，$d$を導入した意味がわかります．
もし$d$が決定的ならば，$t\to t-d$とすればよいですが，$d$を非決定的な場合はこれはできません．
また，このように目的関数に関するデータが非決定的でも，$t$を目的関数にした決定的な問題に還元することができます．

**決定的な目的の場合**

$c$と$d$が決定的な場合は，Robust Counterpartは次の形になります．

$$
\min _x\left\{\widehat{c}(x)=\sup _{(A, b) \in \mathcal{U}}\left[c^T x+d\right]: A x \leq b \forall(A, b) \in \mathcal{U}\right\}
$$

すると，目的と制約が独立しているのがわかります．つまり，制約の行をそれぞれ独立させた

$$
(A x)_i \leq b_i \Leftrightarrow a_i^T x \leq b_i
$$

を考え，そのRobust Counterpartを

$$
a_i^T x \leq b_i \forall\left[a_i ; b_i\right] \in \mathcal{U}_i
$$

とします．ここで，$\mathcal{U}_i=\left\{\left[a_i ; b_i\right]:[A, b] \in \mathcal{U}\right\}$は元の問題の$i$行だけ抜きたしたものです．

すると，元の問題は$\widehat{\mathcal{U}}=\mathcal{U}_1 \times \ldots \times \mathcal{U}_m$　の形を考えても等価になります．

* $x$がロバスト実行可能界ならば，不確実集合の凸包についても実行可能




---




### 不確実線形計画問題への取り組み方

不確実線形計画問題のRobust Counterpartはどのように解けばいいでしょうか．
これを考えるために，次の定義を導入します：

---

集合$X^{+} \subset \mathbb{R}_x^n \times \mathbb{R}_u^k$は次を満たすとき，$X \subset \mathbb{R}_x^n$を「表す」という．

$$
X=\left\{x: \exists u:(x, u) \in X^{+}\right\}
$$

---

これを使うと嬉しい例として，例えば次のような最適化問題を考えてみます．

$$
\min _x\left\{f(x) \text { s.t. } x \text { satisfies } \mathcal{S}_i, i=1, \ldots, m\right\}
$$

ここで，$\mathcal{S}_i$は変数$x$についての制約を持つシステムとします．
一方で，$\mathcal{S}_i^+$を変数$x, v^i$についての制約を持つシステムとして，$\mathcal{S}_i$を表すとします．このとき，

$$
\min _{x, v^1, \ldots, v^m}\left\{f(x) \text { s.t. }\left(x, v^i\right) \text { satisfies } \mathcal{S}_i^{+}, i=1, \ldots, m\right\}
$$

は上の問題と等価です．

これを踏まえて，次のRobust Counterpartを考えてみましょう（目的関数が決定的なら，このような形だけ考えても十分です）

$$
a^T x \leq b \quad \forall\left([a ; b]=\left[a^0 ; b^0\right]+\sum_{\ell=1}^L \zeta_{\ell}\left[a^{\ell} ; b^{\ell}\right]: \zeta \in \mathcal{Z}\right)
$$

---

**簡単な例**

$$
\mathcal{Z}=\operatorname{Box}_1 \equiv\left\{\zeta \in \mathbb{R}^L:\|\zeta\|_{\infty} \leq 1\right\}
$$

を考えます．このとき，

$$
\begin{array}{ccr} 
& {\left[a^0\right]^T x+\sum_{\ell=1}^L \zeta_{\ell}\left[a^{\ell}\right]^T x \leq b^0+\sum_{\ell=1}^L \zeta_{\ell} b^{\ell}} & \forall\left(\zeta:\|\zeta\|_{\infty} \leq 1\right) \\
\Leftrightarrow & \sum_{\ell=1}^L \zeta_{\ell}\left[\left[a^{\ell}\right]^T x-b^{\ell}\right] \leq b^0-\left[a^0\right]^T x \quad \forall\left(\zeta:\left|\zeta_{\ell}\right| \leq 1, \ell=1, \ldots, L\right) \\
\Leftrightarrow \quad & \max _{-1 \leq \zeta_{\ell} \leq 1}\left[\sum_{\ell=1}^L \zeta_{\ell}\left[\left[a^{\ell}\right]^T x-b^{\ell}\right]\right] \leq b^0-\left[a^0\right]^T x &
\end{array}
$$

が成立します．これの最大を与える$\zeta_\ell$は明らかに$\sum_{\ell=1}^L\left|\left[a^l\right]^T x-b^{\ell}\right|$です．

よって，制約としては

$$
\left[a^0\right]^T x+\sum_{\ell=1}^L\left|\left[a^{\ell}\right]^T x-b^{\ell}\right| \leq b^0
$$

を考えれば良いことがわかります．
さらに変形すれば，次の不等式が元の不等式を表現するのがわかります．

$$
\left\{\begin{array}{l}
-u_{\ell} \leq\left[a^{\ell}\right]^T x-b^{\ell} \leq u_{\ell}, \ell=1, \ldots, L \\
{\left[a^0\right]^T x+\sum_{\ell=1}^L u_{\ell} \leq b^0}
\end{array}\right.
$$

これより，元は無限の制約だったものが，$L$個の有限の制約になっていることがわかります．
これを行列形式に変形しましょう．
今回は$x$の変わりに$x'=[x, [u_1, u_2, \cdots, u_L]]$を目的変数とします．

1. $\left[a^{\ell}\right]^T x-b^{\ell} \leq u_{\ell}$の実現

変形して$\left[a^{\ell}\right]^T x - u_{\ell} \leq b^{\ell}$です．
また，$i$番目だけ$1$のベクトル$e_i=[0, 0, \cdots, 1, \cdots, 0]$を考えます．
すると，$a'_\ell=[a^\ell\mid -e_\ell]$とすれば，$[a'_\ell]^T x'\leq b^\ell$で$\ell$番目の制約を満たせます．

2. $-u_{\ell} \leq \left[a^{\ell}\right]^T x-b^{\ell}$の実現

変形して$-\left[a^{\ell}\right]^T x-u_{\ell} \leq -b^{\ell}$なので，$a''_\ell=[-a^\ell \mid -e_\ell]$として，
$[a''_\ell]^T x'\leq -b^\ell$で$\ell$番目の制約を満たせます．

3. ${\left[a^0\right]^T x+\sum_{\ell=1}^L u_{\ell} \leq b^0}$の実現

全部$1$のベクトル$u=[1, 1, \cdots, 1]$を使って，
$a'''_\ell=[a^0\mid u]$とすれば，$[a'''_\ell]^T x'\leq b^0$で制約を満たせます．

試しに簡単な不確実線形計画問題を解いてみましょう．p.7の問題を実装します．


In [75]:
import numpy as np
from scipy.optimize import linprog


c0 = np.array([100, 199.9, -5500, -6100])
d0 = 0.0
b0 = np.array([0.0, 1000, 2000, 800, 100000, 0, 0, 0, 0])
A0 = np.array([
[-0.01, -0.02, 0.500, 0.600],
[1, 1, 0, 0],
[0, 0, 90, 100 ],
[0, 0, 40, 50],
[100, 199.9, 700, 800],
[-1, 0, 0, 0],
[0, -1, 0, 0],
[0, 0, -1, 0],
[0, 0, 0, -1],
])


# ノミナルモデルを先に解いてみます

res = linprog(c0, A_ub=A0, b_ub=b0)
print("ノミナルモデルの解: ", res.fun, res.x)



ノミナルモデルの解:  -8819.657744624841 [  0.         438.78894252  17.5515577    0.        ]


In [97]:
# Robust Counterpartをときます
# 摂動集合は上の簡単な例の場合と同じです．

L = 2
A1 = np.zeros_like(A0)
b1 = np.zeros_like(b0)
A1[0, 0] = 5e-5

A2 = np.zeros_like(A0)
b2 = np.zeros_like(b0)
A2[0, 1] = 4e-4

As = [A1, A2]
bs = [b1, b2]
new_A = []
new_b = []

for l, AB in enumerate(zip(As, bs)):
    A, B = AB
    e = np.zeros(L)
    e[l] = 1.0
    for a, b in zip(A, B):
        # 1の実現
        al1 = np.hstack((a, -e))
        bl1 = b
        new_A.append(al1)
        new_b.append(bl1)

        # 2の実現
        al2 = np.hstack((-a, -e))
        bl2 = -b
        new_A.append(al2)
        new_b.append(bl2)


# 3の実現
for a, b in zip(A0, b0):
    a3 = np.hstack((a, np.ones(L)))
    b3 = b
    new_A.append(a3)
    new_b.append(b3)

new_A = np.array(new_A)
new_b = np.array(new_b)
new_c = np.hstack((c0, np.zeros(L)))

# 最終的に2L+1倍の制約ができます
assert new_A.shape == ((2*L + 1) * A0.shape[0], A0.shape[1] + L)

res = linprog(new_c, A_ub=new_A, b_ub=new_b)
print("ロバストモデルの解: ", res.fun, res.x)

ロバストモデルの解:  -8272.619999790008 [8.77659368e+02 4.39005286e-02 1.74144617e+01 4.39005286e-02
 4.38829684e-02 1.75602114e-05]


教科書のp.10と似た答えは出てますが，ちょっと違いますね...